# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
# from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor

2023-03-13 14:04:33.110520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [2]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data.dropna(axis=0, inplace=True)

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [3]:
# Print the dimensions of data 
data.shape

(41394, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [4]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [5]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed),
                              columns=X_train_cont.columns,
                              index=X_train_cont.index)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_imputed),
                             columns=X_test_cont.columns,
                             index=X_test_cont.index)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [6]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna('missing', axis=0, inplace=True)
X_test_cat.fillna('missing', axis=0, inplace=True)

# OneHotEncode categorical variables
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe,
                                   columns=cat_columns,
                                   index=X_train_cat.index)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe,
                                  columns=cat_columns,
                                  index=X_test_cat.index)

/Users/user/opt/anaconda3/envs/nn/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Run the below cell to combine the numeric and categorical features. 

In [7]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [8]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [9]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [10]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model(), epochs=150, batch_size=256)

2023-03-13 14:06:54.897349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [11]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled)

INFO:tensorflow:Assets written to: ram://3b95103f93e24deeba0ccef28fb92545/assets


INFO:tensorflow:Assets written to: ram://3b95103f93e24deeba0ccef28fb92545/assets
2023-03-13 14:07:05.068022: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://bd189ff07c934b6da6ff4f22c2c56bbd: INVALID_ARGUMENT: ram://bd189ff07c934b6da6ff4f22c2c56bbd is a directory.


Epoch 1/150
91/91 [==============================] - 0s 855us/step - loss: 0.6882 - mse: 0.6882
Epoch 2/150
91/91 [==============================] - 0s 767us/step - loss: 0.2907 - mse: 0.2907
Epoch 3/150
91/91 [==============================] - 0s 727us/step - loss: 0.2466 - mse: 0.2466
Epoch 4/150
91/91 [==============================] - 0s 714us/step - loss: 0.2362 - mse: 0.2362
Epoch 5/150
91/91 [==============================] - 0s 831us/step - loss: 0.2298 - mse: 0.2298
Epoch 6/150
91/91 [==============================] - 0s 768us/step - loss: 0.2253 - mse: 0.2253
Epoch 7/150
91/91 [==============================] - 0s 754us/step - loss: 0.2217 - mse: 0.2217
Epoch 8/150
91/91 [==============================] - 0s 798us/step - loss: 0.2186 - mse: 0.2186
Epoch 9/150
91/91 [==============================] - 0s 670us/step - loss: 0.2160 - mse: 0.2160
Epoch 10/150
91/91 [==============================] - 0s 712us/step - loss: 0.2139 - mse: 0.2139
Epoch 11/150
91/91 [===================

91/91 [==============================] - 0s 722us/step - loss: 0.1924 - mse: 0.1924
Epoch 86/150
91/91 [==============================] - 0s 684us/step - loss: 0.1924 - mse: 0.1924
Epoch 87/150
91/91 [==============================] - 0s 677us/step - loss: 0.1923 - mse: 0.1923
Epoch 88/150
91/91 [==============================] - 0s 654us/step - loss: 0.1921 - mse: 0.1921
Epoch 89/150
91/91 [==============================] - 0s 621us/step - loss: 0.1921 - mse: 0.1921
Epoch 90/150
91/91 [==============================] - 0s 623us/step - loss: 0.1920 - mse: 0.1920
Epoch 91/150
91/91 [==============================] - 0s 648us/step - loss: 0.1920 - mse: 0.1920
Epoch 92/150
91/91 [==============================] - 0s 672us/step - loss: 0.1919 - mse: 0.1919
Epoch 93/150
91/91 [==============================] - 0s 662us/step - loss: 0.1919 - mse: 0.1919
Epoch 94/150
91/91 [==============================] - 0s 634us/step - loss: 0.1917 - mse: 0.1917
Epoch 95/150
91/91 [=======================

INFO:tensorflow:Assets written to: ram://4779d3c3c04248af8c68abde132eb673/assets


...

91/91 [==============================] - 0s 638us/step - loss: 0.1950 - mse: 0.1950
Epoch 86/150
91/91 [==============================] - 0s 637us/step - loss: 0.1948 - mse: 0.1948
Epoch 87/150
91/91 [==============================] - 0s 624us/step - loss: 0.1946 - mse: 0.1946
Epoch 88/150
91/91 [==============================] - 0s 626us/step - loss: 0.1947 - mse: 0.1947
Epoch 89/150
91/91 [==============================] - 0s 672us/step - loss: 0.1946 - mse: 0.1946
Epoch 90/150
91/91 [==============================] - 0s 624us/step - loss: 0.1944 - mse: 0.1944
Epoch 91/150
91/91 [==============================] - 0s 610us/step - loss: 0.1944 - mse: 0.1944
Epoch 92/150
91/91 [==============================] - 0s 633us/step - loss: 0.1945 - mse: 0.1945
Epoch 93/150
91/91 [==============================] - 0s 620us/step - loss: 0.1942 - mse: 0.1942
Epoch 94/150
91/91 [==============================] - 0s 620us/step - loss: 0.1942 - mse: 0.1942
Epoch 95/150
91/91 [=======================

INFO:tensorflow:Assets written to: ram://445bf4e980854cbda56cef87ab9e9b62/assets


INFO:tensorflow:Assets written to: ram://445bf4e980854cbda56cef87ab9e9b62/assets
2023-03-13 14:07:45.781597: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://0ac20d4195634f77ae42f747cc2e52e3: INVALID_ARGUMENT: ram://0ac20d4195634f77ae42f747cc2e52e3 is a directory.


Epoch 1/150
91/91 [==============================] - 0s 758us/step - loss: 0.6829 - mse: 0.6829
Epoch 2/150
91/91 [==============================] - 0s 653us/step - loss: 0.2906 - mse: 0.2906
Epoch 3/150
91/91 [==============================] - 0s 635us/step - loss: 0.2456 - mse: 0.2456
Epoch 4/150
91/91 [==============================] - 0s 624us/step - loss: 0.2354 - mse: 0.2354
Epoch 5/150
91/91 [==============================] - 0s 645us/step - loss: 0.2293 - mse: 0.2293
Epoch 6/150
91/91 [==============================] - 0s 633us/step - loss: 0.2247 - mse: 0.2247
Epoch 7/150
91/91 [==============================] - 0s 622us/step - loss: 0.2212 - mse: 0.2212
Epoch 8/150
91/91 [==============================] - 0s 658us/step - loss: 0.2182 - mse: 0.2182
Epoch 9/150
91/91 [==============================] - 0s 684us/step - loss: 0.2158 - mse: 0.2158
Epoch 10/150
91/91 [==============================] - 0s 622us/step - loss: 0.2137 - mse: 0.2137
Epoch 11/150
91/91 [===================

91/91 [==============================] - 0s 651us/step - loss: 0.1933 - mse: 0.1933
Epoch 86/150
91/91 [==============================] - 0s 634us/step - loss: 0.1931 - mse: 0.1931
Epoch 87/150
91/91 [==============================] - 0s 636us/step - loss: 0.1930 - mse: 0.1930
Epoch 88/150
91/91 [==============================] - 0s 622us/step - loss: 0.1930 - mse: 0.1930
Epoch 89/150
91/91 [==============================] - 0s 678us/step - loss: 0.1928 - mse: 0.1928
Epoch 90/150
91/91 [==============================] - 0s 799us/step - loss: 0.1929 - mse: 0.1929
Epoch 91/150
91/91 [==============================] - 0s 742us/step - loss: 0.1925 - mse: 0.1925
Epoch 92/150
91/91 [==============================] - 0s 753us/step - loss: 0.1927 - mse: 0.1927
Epoch 93/150
91/91 [==============================] - 0s 721us/step - loss: 0.1927 - mse: 0.1927
Epoch 94/150
91/91 [==============================] - 0s 733us/step - loss: 0.1928 - mse: 0.1928
Epoch 95/150
91/91 [=======================

- Find the RMSE on train data 

In [12]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.4379141104386028

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [13]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(y_train, baseline_preds))

4004.8126509952417

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [14]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(20, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(10, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [15]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model(), epochs=200, batch_size=256)

In [16]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled)

INFO:tensorflow:Assets written to: ram://d4c44f2829d34975abc7d83fa385791b/assets


INFO:tensorflow:Assets written to: ram://d4c44f2829d34975abc7d83fa385791b/assets


Epoch 1/200


2023-03-13 14:11:24.751644: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://26cc849c515648c3a0d3d6833b0859e8: INVALID_ARGUMENT: ram://26cc849c515648c3a0d3d6833b0859e8 is a directory.


91/91 [==============================] - 0s 847us/step - loss: 0.4844 - mse: 0.4844
Epoch 2/200
91/91 [==============================] - 0s 800us/step - loss: 0.2722 - mse: 0.2722
Epoch 3/200
91/91 [==============================] - 0s 815us/step - loss: 0.2453 - mse: 0.2453
Epoch 4/200
91/91 [==============================] - 0s 752us/step - loss: 0.2319 - mse: 0.2319
Epoch 5/200
91/91 [==============================] - 0s 732us/step - loss: 0.2235 - mse: 0.2235
Epoch 6/200
91/91 [==============================] - 0s 752us/step - loss: 0.2181 - mse: 0.2181
Epoch 7/200
91/91 [==============================] - 0s 777us/step - loss: 0.2141 - mse: 0.2141
Epoch 8/200
91/91 [==============================] - 0s 728us/step - loss: 0.2112 - mse: 0.2112
Epoch 9/200
91/91 [==============================] - 0s 754us/step - loss: 0.2092 - mse: 0.2092
Epoch 10/200
91/91 [==============================] - 0s 729us/step - loss: 0.2074 - mse: 0.2074
Epoch 11/200
91/91 [==============================]

...

INFO:tensorflow:Assets written to: ram://57d0f66f46bf45f18d09d0625d32d6cf/assets


INFO:tensorflow:Assets written to: ram://57d0f66f46bf45f18d09d0625d32d6cf/assets
2023-03-13 14:12:19.903527: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://2118c1c726884b9b91dfdc49068bd87f: INVALID_ARGUMENT: ram://2118c1c726884b9b91dfdc49068bd87f is a directory.


Epoch 1/200
91/91 [==============================] - 0s 781us/step - loss: 0.4817 - mse: 0.4817
Epoch 2/200
91/91 [==============================] - 0s 675us/step - loss: 0.2711 - mse: 0.2711
Epoch 3/200
91/91 [==============================] - 0s 678us/step - loss: 0.2452 - mse: 0.2452
Epoch 4/200
91/91 [==============================] - 0s 665us/step - loss: 0.2324 - mse: 0.2324
Epoch 5/200
91/91 [==============================] - 0s 689us/step - loss: 0.2247 - mse: 0.2247
Epoch 6/200
91/91 [==============================] - 0s 724us/step - loss: 0.2196 - mse: 0.2196
Epoch 7/200
91/91 [==============================] - 0s 776us/step - loss: 0.2158 - mse: 0.2158
Epoch 8/200
91/91 [==============================] - 0s 712us/step - loss: 0.2128 - mse: 0.2128
Epoch 9/200
91/91 [==============================] - 0s 672us/step - loss: 0.2103 - mse: 0.2103
Epoch 10/200
91/91 [==============================] - 0s 651us/step - loss: 0.2086 - mse: 0.2086
Epoch 11/200
91/91 [===================

91/91 [==============================] - 0s 705us/step - loss: 0.1908 - mse: 0.1908
Epoch 86/200
91/91 [==============================] - 0s 670us/step - loss: 0.1907 - mse: 0.1907
Epoch 87/200
91/91 [==============================] - 0s 702us/step - loss: 0.1906 - mse: 0.1906
Epoch 88/200
91/91 [==============================] - 0s 664us/step - loss: 0.1906 - mse: 0.1906
Epoch 89/200
91/91 [==============================] - 0s 669us/step - loss: 0.1904 - mse: 0.1904
Epoch 90/200
91/91 [==============================] - 0s 682us/step - loss: 0.1904 - mse: 0.1904
Epoch 91/200
91/91 [==============================] - 0s 703us/step - loss: 0.1903 - mse: 0.1903
Epoch 92/200
91/91 [==============================] - 0s 700us/step - loss: 0.1902 - mse: 0.1902
Epoch 93/200
91/91 [==============================] - 0s 705us/step - loss: 0.1900 - mse: 0.1900
Epoch 94/200
91/91 [==============================] - 0s 662us/step - loss: 0.1900 - mse: 0.1900
Epoch 95/200
91/91 [=======================

91/91 [==============================] - 0s 707us/step - loss: 0.1860 - mse: 0.1860
Epoch 169/200
91/91 [==============================] - 0s 692us/step - loss: 0.1857 - mse: 0.1857
Epoch 170/200
91/91 [==============================] - 0s 739us/step - loss: 0.1857 - mse: 0.1857
Epoch 171/200
91/91 [==============================] - 0s 794us/step - loss: 0.1858 - mse: 0.1858
Epoch 172/200
91/91 [==============================] - 0s 680us/step - loss: 0.1856 - mse: 0.1856
Epoch 173/200
91/91 [==============================] - 0s 727us/step - loss: 0.1856 - mse: 0.1856
Epoch 174/200
91/91 [==============================] - 0s 712us/step - loss: 0.1855 - mse: 0.1855
Epoch 175/200
91/91 [==============================] - 0s 713us/step - loss: 0.1856 - mse: 0.1856
Epoch 176/200
91/91 [==============================] - 0s 699us/step - loss: 0.1855 - mse: 0.1855
Epoch 177/200
91/91 [==============================] - 0s 699us/step - loss: 0.1855 - mse: 0.1855
Epoch 178/200
91/91 [=============

In [17]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))

0.43519830187457814

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [18]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(20, activation='relu', input_shape=(n_features,)))
    model.add(layers.Dropout(0.3))

    # Second hidden layer
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [19]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model(), epochs=200, batch_size=256)

In [20]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled)

INFO:tensorflow:Assets written to: ram://b0d30f0a255545f497ffd6f0768c2c9d/assets


INFO:tensorflow:Assets written to: ram://b0d30f0a255545f497ffd6f0768c2c9d/assets
2023-03-13 14:14:22.096332: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://aafe9825afc7414fa7416c383b8c3327: INVALID_ARGUMENT: ram://aafe9825afc7414fa7416c383b8c3327 is a directory.


Epoch 1/200
91/91 [==============================] - 0s 820us/step - loss: 0.6097 - mse: 0.6097
Epoch 2/200
91/91 [==============================] - 0s 886us/step - loss: 0.4719 - mse: 0.4719
Epoch 3/200
91/91 [==============================] - 0s 879us/step - loss: 0.4356 - mse: 0.4356
Epoch 4/200
91/91 [==============================] - 0s 895us/step - loss: 0.4169 - mse: 0.4169
Epoch 5/200
91/91 [==============================] - 0s 822us/step - loss: 0.3926 - mse: 0.3926
Epoch 6/200
91/91 [==============================] - 0s 827us/step - loss: 0.3884 - mse: 0.3884
Epoch 7/200
91/91 [==============================] - 0s 887us/step - loss: 0.3750 - mse: 0.3750
Epoch 8/200
91/91 [==============================] - 0s 825us/step - loss: 0.3704 - mse: 0.3704
Epoch 9/200
91/91 [==============================] - 0s 763us/step - loss: 0.3632 - mse: 0.3632
Epoch 10/200
91/91 [==============================] - 0s 790us/step - loss: 0.3578 - mse: 0.3578
Epoch 11/200
91/91 [===================

91/91 [==============================] - 0s 814us/step - loss: 0.2937 - mse: 0.2937
Epoch 86/200
91/91 [==============================] - 0s 746us/step - loss: 0.2941 - mse: 0.2941
Epoch 87/200
91/91 [==============================] - 0s 777us/step - loss: 0.2977 - mse: 0.2977
Epoch 88/200
91/91 [==============================] - 0s 758us/step - loss: 0.3011 - mse: 0.3011
Epoch 89/200
91/91 [==============================] - 0s 749us/step - loss: 0.2927 - mse: 0.2927
Epoch 90/200
91/91 [==============================] - 0s 742us/step - loss: 0.2931 - mse: 0.2931
Epoch 91/200
91/91 [==============================] - 0s 742us/step - loss: 0.2892 - mse: 0.2892
Epoch 92/200
91/91 [==============================] - 0s 765us/step - loss: 0.2918 - mse: 0.2918
Epoch 93/200
91/91 [==============================] - 0s 726us/step - loss: 0.2887 - mse: 0.2887
Epoch 94/200
91/91 [==============================] - 0s 744us/step - loss: 0.2949 - mse: 0.2949
Epoch 95/200
91/91 [=======================

91/91 [==============================] - 0s 737us/step - loss: 0.2906 - mse: 0.2906
Epoch 169/200
91/91 [==============================] - 0s 756us/step - loss: 0.2870 - mse: 0.2870
Epoch 170/200
91/91 [==============================] - 0s 794us/step - loss: 0.2882 - mse: 0.2882
Epoch 171/200
91/91 [==============================] - 0s 738us/step - loss: 0.2880 - mse: 0.2880
Epoch 172/200
91/91 [==============================] - 0s 732us/step - loss: 0.2853 - mse: 0.2853
Epoch 173/200
91/91 [==============================] - 0s 725us/step - loss: 0.2864 - mse: 0.2864
Epoch 174/200
91/91 [==============================] - 0s 729us/step - loss: 0.2819 - mse: 0.2819
Epoch 175/200
91/91 [==============================] - 0s 754us/step - loss: 0.2902 - mse: 0.2902
Epoch 176/200
91/91 [==============================] - 0s 737us/step - loss: 0.2872 - mse: 0.2872
Epoch 177/200
91/91 [==============================] - 0s 724us/step - loss: 0.2859 - mse: 0.2859
Epoch 178/200
91/91 [=============

...

Epoch 1/200


2023-03-13 14:15:23.752376: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://7bf2dc42b7b24a4eba2eb181260c1bf1: INVALID_ARGUMENT: ram://7bf2dc42b7b24a4eba2eb181260c1bf1 is a directory.


91/91 [==============================] - 0s 782us/step - loss: 0.6203 - mse: 0.6203
Epoch 2/200
91/91 [==============================] - 0s 742us/step - loss: 0.4854 - mse: 0.4854
Epoch 3/200
91/91 [==============================] - 0s 722us/step - loss: 0.4206 - mse: 0.4206
Epoch 4/200
91/91 [==============================] - 0s 770us/step - loss: 0.4084 - mse: 0.4084
Epoch 5/200
91/91 [==============================] - 0s 731us/step - loss: 0.3863 - mse: 0.3863
Epoch 6/200
91/91 [==============================] - 0s 742us/step - loss: 0.3836 - mse: 0.3836
Epoch 7/200
91/91 [==============================] - 0s 736us/step - loss: 0.3840 - mse: 0.3840
Epoch 8/200
91/91 [==============================] - 0s 725us/step - loss: 0.3577 - mse: 0.3577
Epoch 9/200
91/91 [==============================] - 0s 733us/step - loss: 0.3698 - mse: 0.3698
Epoch 10/200
91/91 [==============================] - 0s 739us/step - loss: 0.3647 - mse: 0.3647
Epoch 11/200
91/91 [==============================]

91/91 [==============================] - 0s 854us/step - loss: 0.2909 - mse: 0.2909
Epoch 86/200
91/91 [==============================] - 0s 880us/step - loss: 0.2962 - mse: 0.2962
Epoch 87/200
91/91 [==============================] - 0s 822us/step - loss: 0.2994 - mse: 0.2994
Epoch 88/200
91/91 [==============================] - 0s 802us/step - loss: 0.2896 - mse: 0.2896
Epoch 89/200
91/91 [==============================] - 0s 819us/step - loss: 0.2926 - mse: 0.2926
Epoch 90/200
91/91 [==============================] - 0s 808us/step - loss: 0.2976 - mse: 0.2976
Epoch 91/200
91/91 [==============================] - 0s 853us/step - loss: 0.2899 - mse: 0.2899
Epoch 92/200
91/91 [==============================] - 0s 907us/step - loss: 0.2940 - mse: 0.2940
Epoch 93/200
91/91 [==============================] - 0s 884us/step - loss: 0.2880 - mse: 0.2880
Epoch 94/200
91/91 [==============================] - 0s 889us/step - loss: 0.2909 - mse: 0.2909
Epoch 95/200
91/91 [=======================

Epoch 169/200
91/91 [==============================] - 0s 703us/step - loss: 0.2857 - mse: 0.2857
Epoch 170/200
91/91 [==============================] - 0s 736us/step - loss: 0.2838 - mse: 0.2838
Epoch 171/200
91/91 [==============================] - 0s 781us/step - loss: 0.2832 - mse: 0.2832
Epoch 172/200
91/91 [==============================] - 0s 728us/step - loss: 0.2809 - mse: 0.2809
Epoch 173/200
91/91 [==============================] - 0s 739us/step - loss: 0.2920 - mse: 0.2920
Epoch 174/200
91/91 [==============================] - 0s 816us/step - loss: 0.2860 - mse: 0.2860
Epoch 175/200
91/91 [==============================] - 0s 739us/step - loss: 0.2827 - mse: 0.2827
Epoch 176/200
91/91 [==============================] - 0s 896us/step - loss: 0.2808 - mse: 0.2808
Epoch 177/200
91/91 [==============================] - 0s 748us/step - loss: 0.2867 - mse: 0.2867
Epoch 178/200
91/91 [==============================] - 0s 696us/step - loss: 0.2875 - mse: 0.2875
Epoch 179/200
91/91 

In [21]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

0.46923648262155143

In [27]:
# Define a function that returns a compiled Keras model 
import keras

def create_regularized_model2():
    
    # Initialize model
    model = models.Sequential()
    model.add(keras.Input(shape=n_features,))

    # First hidden layer
    model.add(layers.Dense(50,
                           activation='relu',
                           kernel_regularizer=keras.regularizers.L2(0.005)))
    model.add(layers.Dropout(0.3))

    # Second hidden layer
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [28]:
keras_wrapper_4 = KerasRegressor(create_regularized_model2(), epochs=200, batch_size=256)
cv_l2_preds = cross_val_predict(keras_wrapper_4, X_train_all, y_train_scaled)

INFO:tensorflow:Assets written to: ram://6269a8e5ceb144188a15050954b779cd/assets


INFO:tensorflow:Assets written to: ram://6269a8e5ceb144188a15050954b779cd/assets
2023-03-13 14:21:29.054516: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://8d78022cb5d8411b91f790448902f940: INVALID_ARGUMENT: ram://8d78022cb5d8411b91f790448902f940 is a directory.


Epoch 1/200
91/91 [==============================] - 0s 934us/step - loss: 0.7293 - mse: 0.5863
Epoch 2/200
91/91 [==============================] - 0s 889us/step - loss: 0.5692 - mse: 0.4295
Epoch 3/200
91/91 [==============================] - 0s 939us/step - loss: 0.5285 - mse: 0.3918
Epoch 4/200
91/91 [==============================] - 0s 956us/step - loss: 0.5035 - mse: 0.3697
Epoch 5/200
91/91 [==============================] - 0s 882us/step - loss: 0.4937 - mse: 0.3628
Epoch 6/200
91/91 [==============================] - 0s 912us/step - loss: 0.4789 - mse: 0.3507
Epoch 7/200
91/91 [==============================] - 0s 920us/step - loss: 0.4722 - mse: 0.3466
Epoch 8/200
91/91 [==============================] - 0s 914us/step - loss: 0.4610 - mse: 0.3380
Epoch 9/200
91/91 [==============================] - 0s 872us/step - loss: 0.4560 - mse: 0.3355
Epoch 10/200
91/91 [==============================] - 0s 860us/step - loss: 0.4502 - mse: 0.3321
Epoch 11/200
91/91 [===================

91/91 [==============================] - 0s 859us/step - loss: 0.3000 - mse: 0.2706
Epoch 86/200
91/91 [==============================] - 0s 856us/step - loss: 0.2961 - mse: 0.2673
Epoch 87/200
91/91 [==============================] - 0s 891us/step - loss: 0.3054 - mse: 0.2770
Epoch 88/200
91/91 [==============================] - 0s 855us/step - loss: 0.2957 - mse: 0.2679
Epoch 89/200
91/91 [==============================] - 0s 915us/step - loss: 0.2982 - mse: 0.2708
Epoch 90/200
91/91 [==============================] - 0s 886us/step - loss: 0.2990 - mse: 0.2720
Epoch 91/200
91/91 [==============================] - 0s 907us/step - loss: 0.2951 - mse: 0.2686
Epoch 92/200
91/91 [==============================] - 0s 853us/step - loss: 0.2914 - mse: 0.2653
Epoch 93/200
91/91 [==============================] - 0s 852us/step - loss: 0.2955 - mse: 0.2699
Epoch 94/200
91/91 [==============================] - 0s 957us/step - loss: 0.2971 - mse: 0.2719
Epoch 95/200
91/91 [=======================

91/91 [==============================] - 0s 843us/step - loss: 0.2742 - mse: 0.2657
Epoch 169/200
91/91 [==============================] - 0s 978us/step - loss: 0.2726 - mse: 0.2642
Epoch 170/200
91/91 [==============================] - 0s 909us/step - loss: 0.2737 - mse: 0.2655
Epoch 171/200
91/91 [==============================] - 0s 888us/step - loss: 0.2640 - mse: 0.2559
Epoch 172/200
91/91 [==============================] - 0s 879us/step - loss: 0.2668 - mse: 0.2587
Epoch 173/200
91/91 [==============================] - 0s 835us/step - loss: 0.2709 - mse: 0.2629
Epoch 174/200
91/91 [==============================] - 0s 871us/step - loss: 0.2662 - mse: 0.2584
Epoch 175/200
91/91 [==============================] - 0s 831us/step - loss: 0.2647 - mse: 0.2569
Epoch 176/200
91/91 [==============================] - 0s 869us/step - loss: 0.2766 - mse: 0.2688
Epoch 177/200
91/91 [==============================] - 0s 877us/step - loss: 0.2795 - mse: 0.2719
Epoch 178/200
91/91 [=============

...

Epoch 1/200


2023-03-13 14:22:36.996971: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://4b7466647d8c49b3b789b119f3f04d71: INVALID_ARGUMENT: ram://4b7466647d8c49b3b789b119f3f04d71 is a directory.


91/91 [==============================] - 0s 907us/step - loss: 0.7170 - mse: 0.5740
Epoch 2/200
91/91 [==============================] - 0s 831us/step - loss: 0.5651 - mse: 0.4252
Epoch 3/200
91/91 [==============================] - 0s 835us/step - loss: 0.5350 - mse: 0.3982
Epoch 4/200
91/91 [==============================] - 0s 849us/step - loss: 0.5085 - mse: 0.3746
Epoch 5/200
91/91 [==============================] - 0s 855us/step - loss: 0.4934 - mse: 0.3623
Epoch 6/200
91/91 [==============================] - 0s 821us/step - loss: 0.4798 - mse: 0.3514
Epoch 7/200
91/91 [==============================] - 0s 833us/step - loss: 0.4720 - mse: 0.3463
Epoch 8/200
91/91 [==============================] - 0s 844us/step - loss: 0.4650 - mse: 0.3418
Epoch 9/200
91/91 [==============================] - 0s 847us/step - loss: 0.4546 - mse: 0.3339
Epoch 10/200
91/91 [==============================] - 0s 832us/step - loss: 0.4449 - mse: 0.3266
Epoch 11/200
91/91 [==============================]

91/91 [==============================] - 0s 845us/step - loss: 0.3061 - mse: 0.2768
Epoch 86/200
91/91 [==============================] - 0s 825us/step - loss: 0.2958 - mse: 0.2670
Epoch 87/200
91/91 [==============================] - 0s 823us/step - loss: 0.3036 - mse: 0.2752
Epoch 88/200
91/91 [==============================] - 0s 829us/step - loss: 0.2992 - mse: 0.2714
Epoch 89/200
91/91 [==============================] - 0s 811us/step - loss: 0.2931 - mse: 0.2657
Epoch 90/200
91/91 [==============================] - 0s 865us/step - loss: 0.2935 - mse: 0.2666
Epoch 91/200
91/91 [==============================] - 0s 841us/step - loss: 0.2964 - mse: 0.2699
Epoch 92/200
91/91 [==============================] - 0s 842us/step - loss: 0.2939 - mse: 0.2678
Epoch 93/200
91/91 [==============================] - 0s 813us/step - loss: 0.2908 - mse: 0.2652
Epoch 94/200
91/91 [==============================] - 0s 833us/step - loss: 0.2981 - mse: 0.2730
Epoch 95/200
91/91 [=======================

91/91 [==============================] - 0s 954us/step - loss: 0.2720 - mse: 0.2635
Epoch 169/200
91/91 [==============================] - 0s 896us/step - loss: 0.2724 - mse: 0.2640
Epoch 170/200
91/91 [==============================] - 0s 1ms/step - loss: 0.2711 - mse: 0.2628
Epoch 171/200
91/91 [==============================] - 0s 920us/step - loss: 0.2726 - mse: 0.2644
Epoch 172/200
91/91 [==============================] - 0s 1ms/step - loss: 0.2722 - mse: 0.2642
Epoch 173/200
91/91 [==============================] - 0s 2ms/step - loss: 0.2739 - mse: 0.2659
Epoch 174/200
91/91 [==============================] - 0s 2ms/step - loss: 0.2732 - mse: 0.2653
Epoch 175/200
91/91 [==============================] - 0s 913us/step - loss: 0.2706 - mse: 0.2628
Epoch 176/200
91/91 [==============================] - 0s 882us/step - loss: 0.2704 - mse: 0.2627
Epoch 177/200
91/91 [==============================] - 0s 880us/step - loss: 0.2633 - mse: 0.2557
Epoch 178/200
91/91 [=====================

In [29]:
np.sqrt(mean_squared_error(y_train_scaled, cv_l2_preds))

0.4427503805468388

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [31]:
# ⏰ This cell may take several mintes to run
best_model = create_bigger_model()
best_model.fit(X_train_all,
               y_train_scaled,
               epochs=150,
               batch_size=256,
               verbose=0)

In [34]:
# un-scale predictions, to increase interpretability

y_test_preds_scaled = best_model.predict(X_test_all)

y_test_preds = ss_y.inverse_transform(y_test_preds_scaled)

np.sqrt(mean_squared_error(y_test, y_test_preds))

389/389 [==============================] - 0s 479us/step


3926.0490515659467

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 